# NumPy for Numerical Computing

```{admonition} Information
:class: info

**Prerequisites:** Module 02 Lesson 1 (Python Fundamentals)  
**Learning Objectives:**
- Create and manipulate NumPy arrays for engineering data
- Perform vectorized operations for efficient calculations
- Apply statistical analysis to measurement data
- Work with multi-dimensional arrays and matrices
- Understand performance benefits of NumPy over pure Python

**Estimated Time:** 2 hours
```

## Introduction

NumPy is the foundation of scientific computing in Python, providing efficient array operations that are essential for engineering analysis. Unlike Python lists, NumPy arrays enable vectorized operations that make mathematical computations both faster and clearer. This lesson demonstrates NumPy fundamentals using power system data as context, focusing on measurement processing, statistical analysis, and basic calculations.

Let's begin by comparing a simple electrical calculation using pure Python versus NumPy to understand the performance and clarity benefits.

In [1]:
import numpy as np
import time

# Power calculation for multiple loads (P = V * I * cos(φ))
# Pure Python approach
def power_calculation_python(voltage_list, current_list, pf_list):
    """Calculate power using pure Python"""
    powers = []
    for i in range(len(voltage_list)):
        power = voltage_list[i] * current_list[i] * pf_list[i]
        powers.append(power)
    return powers

# NumPy approach
def power_calculation_numpy(voltage, current, pf):
    """Calculate power using NumPy"""
    return voltage * current * pf

# Create test data for 1000 electrical loads
n_loads = 1000
voltage = np.random.uniform(0.95, 1.05, n_loads) * 230  # Voltage in V
current = np.random.uniform(10, 100, n_loads)  # Current in A
pf = np.random.uniform(0.8, 0.95, n_loads)  # Power factor

# Convert to lists for Python version
voltage_list = voltage.tolist()
current_list = current.tolist()
pf_list = pf.tolist()

# Time comparison
start = time.time()
powers_python = power_calculation_python(voltage_list, current_list, pf_list)
python_time = time.time() - start

start = time.time()
powers_numpy = power_calculation_numpy(voltage, current, pf)
numpy_time = time.time() - start

print(f"Pure Python time: {python_time*1000:.2f} ms")
print(f"NumPy time: {numpy_time*1000:.2f} ms")
print(f"Speed improvement: {python_time/numpy_time:.1f}x")
print(f"\nResults match: {np.allclose(powers_python, powers_numpy)}")

Pure Python time: 0.17 ms
NumPy time: 0.09 ms
Speed improvement: 1.9x

Results match: True


## 1. NumPy Arrays and Power System Data

NumPy arrays are the natural representation for power system measurements and data. Sensor readings, time series data, and measurement matrices all map directly to NumPy's n-dimensional arrays.

### Creating Arrays for Measurements

In [2]:
# Voltage measurement data from 5 monitoring points
monitor_names = ['Station_A', 'Station_B', 'Station_C', 'Station_D', 'Station_E']
# Voltage measurements in kV (nominal 230 kV)
voltage_readings = np.array([232.5, 228.9, 231.2, 229.8, 233.1])

print("Voltage Measurement Data:")
print(f"Readings (kV): {voltage_readings}")
print(f"\nArray properties:")
print(f"Shape: {voltage_readings.shape}")
print(f"Data type: {voltage_readings.dtype}")
print(f"Memory usage: {voltage_readings.nbytes} bytes")

# Convert to per-unit (base = 230 kV)
base_voltage = 230.0
voltage_pu = voltage_readings / base_voltage
print(f"\nPer-unit values: {voltage_pu}")
print(f"Average: {voltage_pu.mean():.4f} pu")
print(f"Standard deviation: {voltage_pu.std():.4f} pu")
print(f"Range: {voltage_pu.min():.4f} - {voltage_pu.max():.4f} pu")

# Check if voltages are within acceptable limits (±5%)
within_limits = (voltage_pu >= 0.95) & (voltage_pu <= 1.05)
print(f"\nVoltages within limits: {within_limits}")
print(f"All voltages acceptable: {within_limits.all()}")

Voltage Measurement Data:
Readings (kV): [232.5 228.9 231.2 229.8 233.1]

Array properties:
Shape: (5,)
Data type: float64
Memory usage: 40 bytes

Per-unit values: [1.01086957 0.99521739 1.00521739 0.99913043 1.01347826]
Average: 1.0048 pu
Standard deviation: 0.0069 pu
Range: 0.9952 - 1.0135 pu

Voltages within limits: [ True  True  True  True  True]
All voltages acceptable: True


### Working with Time Series Data

In [3]:
# Load data for 24 hours (hourly readings)
hours = np.arange(24)

# Simulate daily load pattern
base_load = 150  # MW
daily_pattern = base_load * (1 + 0.3 * np.sin((hours - 6) * np.pi / 12))
# Add some random variation
load_data = daily_pattern + np.random.normal(0, 5, 24)

print("Daily Load Profile:")
print(f"Hour  Load (MW)")
print("=" * 20)
for h, load in enumerate(load_data[:6]):  # Show first 6 hours
    print(f"{h:4d}  {load:8.1f}")
print("...\n")

# Statistical analysis
print("Daily Statistics:")
print(f"Average load: {load_data.mean():.1f} MW")
print(f"Peak load: {load_data.max():.1f} MW at hour {load_data.argmax()}")
print(f"Minimum load: {load_data.min():.1f} MW at hour {load_data.argmin()}")
print(f"Load factor: {load_data.mean() / load_data.max():.3f}")

# Calculate energy consumed
energy = load_data.sum()  # MWh (since hourly data)
print(f"\nTotal daily energy: {energy:.0f} MWh")

Daily Load Profile:
Hour  Load (MW)
   0     103.0
   1     106.2
   2     101.5
   3     118.7
   4     125.9
   5     141.6
...

Daily Statistics:
Average load: 148.9 MW
Peak load: 198.4 MW at hour 12
Minimum load: 101.5 MW at hour 2
Load factor: 0.751

Total daily energy: 3574 MWh


### Multi-dimensional Arrays

In [4]:
# Create a 2D array: 7 days × 24 hours of load data
# Each row is one day, each column is one hour
weekly_load = np.zeros((7, 24))

# Generate weekly pattern
for day in range(7):
    # Weekday vs weekend
    if day < 5:  # Monday-Friday
        base = 150
    else:  # Weekend
        base = 120
    
    daily = base * (1 + 0.3 * np.sin((hours - 6) * np.pi / 12))
    weekly_load[day] = daily + np.random.normal(0, 5, 24)

print(f"Weekly load data shape: {weekly_load.shape}")
print(f"Total size: {weekly_load.size} elements\n")

# Analysis by dimension
daily_average = weekly_load.mean(axis=1)  # Average across hours
hourly_average = weekly_load.mean(axis=0)  # Average across days

print("Average load by day:")
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for day, avg in zip(days, daily_average):
    print(f"{day}: {avg:6.1f} MW")

print("\nPeak hour analysis:")
peak_hour_avg = hourly_average.argmax()
print(f"Peak typically occurs at hour {peak_hour_avg}")
print(f"Average peak value: {hourly_average[peak_hour_avg]:.1f} MW")

Weekly load data shape: (7, 24)
Total size: 168 elements

Average load by day:
Mon:  150.2 MW
Tue:  149.1 MW
Wed:  148.9 MW
Thu:  149.0 MW
Fri:  149.8 MW
Sat:  120.5 MW
Sun:  120.2 MW

Peak hour analysis:
Peak typically occurs at hour 13
Average peak value: 183.8 MW


```{admonition} Exercise 1: Array Creation and Basic Operations
:class: dropdown

Create NumPy arrays for power system measurements:
- Voltage readings: [11.2, 10.9, 11.1, 10.8, 11.3] kV (nominal 11 kV)
- Current readings: [250, 180, 220, 195, 240] A
- Power factor: [0.85, 0.90, 0.88, 0.92, 0.86]

Calculate:
1. Convert voltages to per-unit (base = 11 kV)
2. Calculate apparent power S = V × I for each measurement
3. Calculate active power P = S × PF
4. Find the total active power and average power factor

```

In [5]:
# Solution to Exercise 1
# Measurement data
voltage_kv = np.array([11.2, 10.9, 11.1, 10.8, 11.3])  # kV
current_a = np.array([250, 180, 220, 195, 240])  # A
power_factor = np.array([0.85, 0.90, 0.88, 0.92, 0.86])

# 1. Convert to per-unit
base_kv = 11.0
voltage_pu = voltage_kv / base_kv
print(f"Voltages in per-unit: {voltage_pu}")
print(f"Voltage range: {voltage_pu.min():.3f} - {voltage_pu.max():.3f} pu")

# 2. Calculate apparent power S = V × I
apparent_power_kva = voltage_kv * current_a  # kVA
apparent_power_mva = apparent_power_kva / 1000  # MVA
print(f"\nApparent power (MVA): {apparent_power_mva}")

# 3. Calculate active power P = S × PF
active_power_mw = apparent_power_mva * power_factor
print(f"\nActive power (MW): {active_power_mw}")

# 4. Total and average
total_active_power = active_power_mw.sum()
avg_power_factor = (active_power_mw.sum()) / (apparent_power_mva.sum())

print(f"\nTotal active power: {total_active_power:.2f} MW")
print(f"Average power factor: {avg_power_factor:.3f}")

# Additional: Calculate reactive power
reactive_power_mvar = apparent_power_mva * np.sqrt(1 - power_factor**2)
print(f"\nReactive power (MVAR): {reactive_power_mvar}")
print(f"Total reactive power: {reactive_power_mvar.sum():.2f} MVAR")

Voltages in per-unit: [1.01818182 0.99090909 1.00909091 0.98181818 1.02727273]
Voltage range: 0.982 - 1.027 pu

Apparent power (MVA): [2.8   1.962 2.442 2.106 2.712]

Active power (MW): [2.38    1.7658  2.14896 1.93752 2.33232]

Total active power: 10.56 MW
Average power factor: 0.879

Reactive power (MVAR): [1.47499153 0.85521597 1.15988574 0.82538006 1.38391742]
Total reactive power: 5.70 MVAR


## 2. Array Operations and Broadcasting

NumPy's strength lies in its ability to perform operations on entire arrays without explicit loops. Broadcasting allows operations between arrays of different shapes.

### Vectorized Operations

In [6]:
# Temperature effect on resistance
# R(T) = R0 * (1 + α * (T - T0))
base_resistance = np.array([0.5, 0.8, 0.6, 0.7, 0.9])  # Ohms at 20°C
alpha = 0.004  # Temperature coefficient (1/°C)
base_temp = 20  # °C

# Temperature readings throughout the day
temperatures = np.array([15, 18, 22, 28, 35, 32, 25, 20])  # °C

# Calculate resistance at each temperature for all conductors
# Using broadcasting: (5,1) × (8,) -> (5,8)
resistance_matrix = base_resistance[:, np.newaxis] * (1 + alpha * (temperatures - base_temp))

print("Resistance variation with temperature:")
print("Temp(°C):", temperatures)
print("\nResistance values (Ohms):")
for i, r0 in enumerate(base_resistance):
    print(f"R0={r0:.1f}: {resistance_matrix[i]}")

# Calculate average resistance for each conductor
avg_resistance = resistance_matrix.mean(axis=1)
print(f"\nAverage resistance: {avg_resistance}")

# Find maximum resistance increase
max_increase = (resistance_matrix.max(axis=1) - base_resistance) / base_resistance * 100
print(f"Maximum resistance increase: {max_increase}%")

Resistance variation with temperature:
Temp(°C): [15 18 22 28 35 32 25 20]

Resistance values (Ohms):
R0=0.5: [0.49  0.496 0.504 0.516 0.53  0.524 0.51  0.5  ]
R0=0.8: [0.784  0.7936 0.8064 0.8256 0.848  0.8384 0.816  0.8   ]
R0=0.6: [0.588  0.5952 0.6048 0.6192 0.636  0.6288 0.612  0.6   ]
R0=0.7: [0.686  0.6944 0.7056 0.7224 0.742  0.7336 0.714  0.7   ]
R0=0.9: [0.882  0.8928 0.9072 0.9288 0.954  0.9432 0.918  0.9   ]

Average resistance: [0.50875 0.814   0.6105  0.71225 0.91575]
Maximum resistance increase: [6. 6. 6. 6. 6.]%


### Conditional Operations

In [7]:
# Power quality analysis: identify and correct outliers
# Voltage measurements with some bad data
voltage_samples = np.array([230.5, 228.9, 231.2, 195.0, 233.1, 229.8, 
                           280.0, 230.2, 231.5, 228.7])  # kV

print(f"Original data: {voltage_samples}")

# Identify outliers (more than 10% from nominal)
nominal = 230.0
lower_limit = nominal * 0.9
upper_limit = nominal * 1.1

outliers = (voltage_samples < lower_limit) | (voltage_samples > upper_limit)
print(f"\nOutlier mask: {outliers}")
print(f"Outlier values: {voltage_samples[outliers]}")
print(f"Outlier indices: {np.where(outliers)[0]}")

# Replace outliers with median of good values
good_values = voltage_samples[~outliers]
median_value = np.median(good_values)
print(f"\nMedian of good values: {median_value:.1f} kV")

# Create cleaned data
cleaned_data = np.where(outliers, median_value, voltage_samples)
print(f"Cleaned data: {cleaned_data}")

# Statistics comparison
print(f"\nOriginal mean: {voltage_samples.mean():.1f} kV")
print(f"Cleaned mean: {cleaned_data.mean():.1f} kV")
print(f"Original std: {voltage_samples.std():.1f} kV")
print(f"Cleaned std: {cleaned_data.std():.1f} kV")

Original data: [230.5 228.9 231.2 195.  233.1 229.8 280.  230.2 231.5 228.7]

Outlier mask: [False False False  True False False  True False False False]
Outlier values: [195. 280.]
Outlier indices: [3 6]

Median of good values: 230.3 kV
Cleaned data: [230.5  228.9  231.2  230.35 233.1  229.8  230.35 230.2  231.5  228.7 ]

Original mean: 231.9 kV
Cleaned mean: 230.5 kV
Original std: 19.3 kV
Cleaned std: 1.2 kV


```{admonition} Exercise 2: Array Operations and Analysis
:class: dropdown

Given hourly energy consumption data for 5 buildings over 24 hours:
1. Create a 5×24 array with random consumption between 50-200 kWh
2. Calculate total daily consumption for each building
3. Find the peak hour for the entire system
4. Calculate the load factor for each building
5. Identify hours when total system load exceeds 600 kWh

```

In [8]:
# Solution to Exercise 2
np.random.seed(42)  # For reproducibility

# 1. Create consumption data
buildings = 5
hours = 24
consumption = np.random.uniform(50, 200, (buildings, hours))  # kWh

print(f"Consumption data shape: {consumption.shape}")
print(f"First building, first 6 hours: {consumption[0, :6]}")

# 2. Total daily consumption per building
daily_total = consumption.sum(axis=1)
print(f"\nDaily consumption by building (kWh):")
for i, total in enumerate(daily_total):
    print(f"Building {i+1}: {total:,.0f}")

# 3. Find peak hour for system
hourly_system_total = consumption.sum(axis=0)
peak_hour = hourly_system_total.argmax()
peak_load = hourly_system_total[peak_hour]

print(f"\nSystem peak: {peak_load:.0f} kWh at hour {peak_hour}")

# 4. Load factor for each building
# Load factor = average load / peak load
building_peaks = consumption.max(axis=1)
building_average = consumption.mean(axis=1)
load_factors = building_average / building_peaks

print(f"\nLoad factors by building:")
for i, lf in enumerate(load_factors):
    print(f"Building {i+1}: {lf:.3f}")

# 5. Hours exceeding 600 kWh
high_load_hours = np.where(hourly_system_total > 600)[0]
print(f"\nHours with system load > 600 kWh: {high_load_hours}")
print(f"Number of high load hours: {len(high_load_hours)}")

# Additional analysis
print(f"\nSystem statistics:")
print(f"Average hourly load: {hourly_system_total.mean():.0f} kWh")
print(f"System load factor: {hourly_system_total.mean() / peak_load:.3f}")

Consumption data shape: (5, 24)
First building, first 6 hours: [106.18101783 192.60714596 159.79909127 139.79877263  73.40279607
  73.39917805]

Daily consumption by building (kWh):
Building 1: 2,785
Building 2: 2,850
Building 3: 3,020
Building 4: 3,036
Building 5: 2,898

System peak: 788 kWh at hour 19

Load factors by building:
Building 1: 0.594
Building 2: 0.609
Building 3: 0.635
Building 4: 0.691
Building 5: 0.637

Hours with system load > 600 kWh: [ 1  2  3  4  7  8  9 11 14 16 17 19 20 21 22]
Number of high load hours: 15

System statistics:
Average hourly load: 608 kWh
System load factor: 0.771


## 3. Working with Complex Numbers

Complex numbers are essential for AC circuit analysis. NumPy provides native support for complex arithmetic.

In [9]:
# Complex power calculations
# S = P + jQ (apparent power = active + j*reactive)

# Power measurements at different locations
active_power = np.array([100, 150, 80, 120, 90])  # MW
reactive_power = np.array([50, 30, 60, 40, 45])   # MVAR

# Create complex power
complex_power = active_power + 1j * reactive_power

print("Complex Power Analysis:")
print("Location  P(MW)  Q(MVAR)  |S|(MVA)  PF     Angle(°)")
print("=" * 55)

for i in range(len(complex_power)):
    s = complex_power[i]
    magnitude = abs(s)
    angle = np.angle(s, deg=True)
    pf = active_power[i] / magnitude
    
    print(f"   {i+1}      {s.real:5.0f}    {s.imag:5.0f}    {magnitude:6.1f}   "
          f"{pf:.3f}   {angle:5.1f}")

# Total system power
total_complex = complex_power.sum()
print(f"\nTotal: S = {total_complex} MVA")
print(f"       = {abs(total_complex):.1f}∠{np.angle(total_complex, deg=True):.1f}° MVA")

# Power factor correction calculation
# To improve PF to 0.95 lagging
target_pf = 0.95
target_angle = np.arccos(target_pf)
target_q = active_power * np.tan(target_angle)
q_compensation = reactive_power - target_q

print("\nPower Factor Correction to 0.95:")
for i in range(len(active_power)):
    print(f"Location {i+1}: {q_compensation[i]:6.1f} MVAR capacitor needed")

Complex Power Analysis:
Location  P(MW)  Q(MVAR)  |S|(MVA)  PF     Angle(°)
   1        100       50     111.8   0.894    26.6
   2        150       30     153.0   0.981    11.3
   3         80       60     100.0   0.800    36.9
   4        120       40     126.5   0.949    18.4
   5         90       45     100.6   0.894    26.6

Total: S = (540+225j) MVA
       = 585.0∠22.6° MVA

Power Factor Correction to 0.95:
Location 1:   17.1 MVAR capacitor needed
Location 2:  -19.3 MVAR capacitor needed
Location 3:   33.7 MVAR capacitor needed
Location 4:    0.6 MVAR capacitor needed
Location 5:   15.4 MVAR capacitor needed


## 4. Matrix Operations

Matrix operations are fundamental to many engineering calculations. NumPy provides efficient implementations of linear algebra operations.

### Basic Matrix Operations

In [10]:
# Resistance matrix for a network (simplified)
# Diagonal: self-resistance, Off-diagonal: mutual resistance
R = np.array([
    [0.1, 0.02, 0.01],
    [0.02, 0.15, 0.03],
    [0.01, 0.03, 0.12]
])

print("Resistance matrix (Ω):")
print(R)

# Current vector
I = np.array([10, 15, 12])  # Amperes

# Calculate voltage drops: V = R × I
V = R @ I  # Matrix multiplication
print(f"\nCurrent vector (A): {I}")
print(f"Voltage drops (V): {V}")

# Power dissipation: P = I^T × V
P_total = I @ V  # Dot product
print(f"\nTotal power dissipation: {P_total:.1f} W")

# Individual power dissipation
P_individual = I * V
print(f"Power by element: {P_individual} W")

# Matrix properties
print(f"\nMatrix properties:")
print(f"Determinant: {np.linalg.det(R):.4f}")
print(f"Condition number: {np.linalg.cond(R):.2f}")
print(f"Symmetric: {np.allclose(R, R.T)}")

Resistance matrix (Ω):
[[0.1  0.02 0.01]
 [0.02 0.15 0.03]
 [0.01 0.03 0.12]]

Current vector (A): [10 15 12]
Voltage drops (V): [1.42 2.81 1.99]

Total power dissipation: 80.2 W
Power by element: [14.2  42.15 23.88] W

Matrix properties:
Determinant: 0.0017
Condition number: 1.88
Symmetric: True


### Solving Linear Systems

In [11]:
# Solve for currents given voltages and impedances
# V = Z × I, solve for I

# Impedance matrix (simplified 3-node network)
Z = np.array([
    [0.1+0.3j, 0.05+0.1j, 0.02+0.05j],
    [0.05+0.1j, 0.15+0.4j, 0.03+0.08j],
    [0.02+0.05j, 0.03+0.08j, 0.12+0.35j]
], dtype=complex)

# Voltage sources
V_sources = np.array([230+0j, 225-5j, 220-10j], dtype=complex)  # V

# Solve for currents
I_solved = np.linalg.solve(Z, V_sources)

print("Network Solution:")
print("Node  Voltage (V)           Current (A)           |I| (A)")
print("=" * 60)
for i in range(3):
    print(f" {i+1}    {V_sources[i]:12}     {I_solved[i]:12}     {abs(I_solved[i]):6.1f}")

# Verify solution: V_check = Z × I
V_check = Z @ I_solved
print(f"\nVerification (should match source voltages):")
print(f"V_check: {V_check}")
print(f"Error: {np.max(np.abs(V_check - V_sources)):.2e}")

# Power calculation
S = V_sources * np.conj(I_solved)
print(f"\nComplex power at each node:")
for i in range(3):
    print(f"Node {i+1}: P = {S[i].real:6.1f} W, Q = {S[i].imag:6.1f} VAR")

Network Solution:
Node  Voltage (V)           Current (A)           |I| (A)
 1        (230+0j)     (187.25165835757147-514.5030217577109j)      547.5
 2        (225-5j)     (115.54561509922269-271.9104162864128j)      295.4
 3       (220-10j)     (118.93113921084279-431.53964938220633j)      447.6

Verification (should match source voltages):
V_check: [230.-1.95399252e-14j 225.-5.00000000e+00j 220.-1.00000000e+01j]
Error: 1.95e-14

Complex power at each node:
Node 1: P = 43067.9 W, Q = 118335.7 VAR
Node 2: P = 27357.3 W, Q = 60602.1 VAR
Node 3: P = 30480.2 W, Q = 93749.4 VAR


```{admonition} Exercise 3: Matrix Operations
:class: dropdown

A simple DC circuit has the following conductance matrix G (in Siemens):
```
G = [[5, -2, -1],
     [-2, 6, -3],
     [-1, -3, 7]]
```
Current injections at each node are: I = [10, -5, -5] A

1. Calculate the node voltages using V = G^(-1) × I
2. Verify the solution by computing I_check = G × V
3. Calculate power at each node P = V × I
4. Check power balance (sum should be zero)

```

In [12]:
# Solution to Exercise 3
# Conductance matrix
G = np.array([
    [5, -2, -1],
    [-2, 6, -3],
    [-1, -3, 7]
])

# Current injections
I = np.array([10, -5, -5])

print("Given:")
print(f"Conductance matrix G (S):\n{G}")
print(f"Current injections I (A): {I}")

# 1. Solve for voltages
V = np.linalg.solve(G, I)
print(f"\n1. Node voltages V (V): {V}")

# 2. Verify solution
I_check = G @ V
print(f"\n2. Verification I_check = G × V: {I_check}")
print(f"   Error: {np.max(np.abs(I_check - I)):.2e}")

# 3. Calculate power
P = V * I
print(f"\n3. Power at each node (W):")
for i in range(3):
    print(f"   Node {i+1}: {P[i]:8.2f} W {'(injected)' if P[i] > 0 else '(absorbed)'}")

# 4. Power balance
P_total = P.sum()
print(f"\n4. Total power: {P_total:.6f} W")
print(f"   Power balance {'OK' if abs(P_total) < 1e-10 else 'ERROR'}!")

# Additional: Calculate resistance matrix R = G^(-1)
R = np.linalg.inv(G)
print(f"\nResistance matrix R = G^(-1) (Ω):")
print(R)

Given:
Conductance matrix G (S):
[[ 5 -2 -1]
 [-2  6 -3]
 [-1 -3  7]]
Current injections I (A): [10 -5 -5]

1. Node voltages V (V): [ 1.55462185 -0.71428571 -0.79831933]

2. Verification I_check = G × V: [10. -5. -5.]
   Error: 8.88e-16

3. Power at each node (W):
   Node 1:    15.55 W (injected)
   Node 2:     3.57 W (injected)
   Node 3:     3.99 W (injected)

4. Total power: 23.109244 W
   Power balance ERROR!

Resistance matrix R = G^(-1) (Ω):
[[0.27731092 0.14285714 0.10084034]
 [0.14285714 0.28571429 0.14285714]
 [0.10084034 0.14285714 0.21848739]]


## 5. Data Processing and Analysis

NumPy provides powerful tools for processing and analyzing large datasets efficiently.

### Signal Processing Basics

In [13]:
# Analyze harmonic content in a voltage signal
# Generate a distorted voltage signal
t = np.linspace(0, 0.1, 1000)  # 0.1 seconds, 1000 samples
f_fundamental = 50  # Hz

# Voltage with harmonics
v_fundamental = 230 * np.sqrt(2) * np.sin(2 * np.pi * f_fundamental * t)
v_3rd = 10 * np.sqrt(2) * np.sin(2 * np.pi * 3 * f_fundamental * t)
v_5th = 7 * np.sqrt(2) * np.sin(2 * np.pi * 5 * f_fundamental * t)
v_total = v_fundamental + v_3rd + v_5th

# Calculate RMS values using different methods
# Method 1: Direct calculation
v_rms_direct = np.sqrt(np.mean(v_total**2))

# Method 2: Using individual harmonics
v_rms_fund = 230
v_rms_3rd = 10
v_rms_5th = 7
v_rms_calc = np.sqrt(v_rms_fund**2 + v_rms_3rd**2 + v_rms_5th**2)

print("Harmonic Analysis:")
print(f"Fundamental: {v_rms_fund} V RMS")
print(f"3rd harmonic: {v_rms_3rd} V RMS ({v_rms_3rd/v_rms_fund*100:.1f}%)")
print(f"5th harmonic: {v_rms_5th} V RMS ({v_rms_5th/v_rms_fund*100:.1f}%)")
print(f"\nTotal RMS (direct): {v_rms_direct:.1f} V")
print(f"Total RMS (calculated): {v_rms_calc:.1f} V")

# Calculate THD
thd = np.sqrt(v_rms_3rd**2 + v_rms_5th**2) / v_rms_fund * 100
print(f"\nTotal Harmonic Distortion: {thd:.1f}%")

# Find peaks and zero crossings
# Positive zero crossings
zero_crossings = np.where(np.diff(np.sign(v_total)))[0]
positive_crossings = zero_crossings[::2]  # Every other one
period_samples = np.diff(positive_crossings)
frequency_measured = 1 / (period_samples.mean() / 1000 * 0.1)  # Sampling rate consideration

print(f"\nMeasured frequency: {frequency_measured:.2f} Hz")

Harmonic Analysis:
Fundamental: 230 V RMS
3rd harmonic: 10 V RMS (4.3%)
5th harmonic: 7 V RMS (3.0%)

Total RMS (direct): 230.2 V
Total RMS (calculated): 230.3 V

Total Harmonic Distortion: 5.3%

Measured frequency: 50.10 Hz


### Data Aggregation and Grouping

In [14]:
# Analyze energy consumption by category
# Simulate monthly consumption data for different sectors
months = 12
sectors = ['Residential', 'Commercial', 'Industrial', 'Transportation']

# Base consumption patterns (GWh)
base_consumption = np.array([300, 250, 400, 150])

# Seasonal variations
seasonal_factors = np.array([
    1.2, 1.15, 1.0, 0.9, 0.8, 0.85, 0.9, 0.95, 0.85, 0.9, 1.0, 1.1  # Monthly factors
])

# Generate consumption matrix
consumption_data = base_consumption[:, np.newaxis] * seasonal_factors
# Add random variation
consumption_data += np.random.normal(0, 10, (4, 12))

print("Monthly Energy Consumption (GWh):")
print("Month", end="")
for s in sectors:
    print(f"\t{s[:4]}", end="")
print("\tTotal")
print("=" * 50)

monthly_totals = consumption_data.sum(axis=0)
for m in range(months):
    print(f"{m+1:2d}", end="")
    for s in range(4):
        print(f"\t{consumption_data[s, m]:.0f}", end="")
    print(f"\t{monthly_totals[m]:.0f}")

# Annual statistics
print("\nAnnual Summary:")
annual_by_sector = consumption_data.sum(axis=1)
for s, sector in enumerate(sectors):
    percentage = annual_by_sector[s] / annual_by_sector.sum() * 100
    print(f"{sector}: {annual_by_sector[s]:.0f} GWh ({percentage:.1f}%)")

print(f"\nTotal annual consumption: {annual_by_sector.sum():.0f} GWh")

# Find peak months by sector
print("\nPeak consumption months:")
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for s, sector in enumerate(sectors):
    peak_month = consumption_data[s].argmax()
    print(f"{sector}: {month_names[peak_month]} ({consumption_data[s, peak_month]:.0f} GWh)")

Monthly Energy Consumption (GWh):
Month	Resi	Comm	Indu	Tran	Total
 1	353	298	480	181	1312
 2	342	292	448	167	1249
 3	296	269	411	134	1111
 4	255	227	368	136	985
 5	243	203	328	109	883
 6	258	212	331	132	933
 7	270	206	374	126	976
 8	283	237	366	158	1044
 9	241	213	346	120	919
10	266	250	382	132	1029
11	297	248	390	158	1093
12	322	278	434	153	1187

Annual Summary:
Residential: 3425 GWh (26.9%)
Commercial: 2932 GWh (23.0%)
Industrial: 4658 GWh (36.6%)
Transportation: 1706 GWh (13.4%)

Total annual consumption: 12721 GWh

Peak consumption months:
Residential: Jan (353 GWh)
Commercial: Jan (298 GWh)
Industrial: Jan (480 GWh)
Transportation: Jan (181 GWh)


```{admonition} Exercise 4: Comprehensive Analysis
:class: dropdown

A monitoring system records voltage and current at 5-minute intervals for 24 hours.
Generate synthetic data and perform the following analysis:

1. Create voltage data: 230V nominal with ±2% random variation
2. Create current data: base pattern with morning and evening peaks
3. Calculate instantaneous power for each interval
4. Find total energy consumed (kWh)
5. Identify periods where power exceeds 50 kW
6. Calculate the cost at $0.12/kWh

```

In [15]:
# Solution to Exercise 4
# Time points: 24 hours × 12 intervals/hour = 288 points
n_points = 24 * 12
time_hours = np.linspace(0, 24, n_points)

# 1. Generate voltage data
v_nominal = 230
voltage = v_nominal * (1 + np.random.uniform(-0.02, 0.02, n_points))

# 2. Generate current with daily pattern
# Base load + morning peak (7-9) + evening peak (18-21)
base_current = 100  # A
current = base_current * np.ones(n_points)

# Add peaks
for i, t in enumerate(time_hours):
    if 7 <= t <= 9:  # Morning peak
        current[i] *= 1.8
    elif 18 <= t <= 21:  # Evening peak
        current[i] *= 2.2
    elif 0 <= t <= 6 or t >= 22:  # Night time
        current[i] *= 0.6

# Add random variation
current += np.random.normal(0, 5, n_points)

# 3. Calculate power (assuming unity power factor)
power_w = voltage * current  # Watts
power_kw = power_w / 1000   # kW

print("Power System Monitoring Analysis")
print("=" * 40)
print(f"Monitoring period: 24 hours")
print(f"Sample interval: 5 minutes")
print(f"Total samples: {n_points}")

# 4. Calculate energy
# Energy = Power × Time (5 minutes = 1/12 hour)
energy_kwh = power_kw * (1/12)  # kWh for each interval
total_energy = energy_kwh.sum()

print(f"\nTotal energy consumed: {total_energy:.1f} kWh")
print(f"Average power: {power_kw.mean():.1f} kW")
print(f"Peak power: {power_kw.max():.1f} kW at hour {time_hours[power_kw.argmax()]:.1f}")

# 5. Identify high power periods
high_power_mask = power_kw > 50
high_power_indices = np.where(high_power_mask)[0]
high_power_hours = time_hours[high_power_indices]

print(f"\nPeriods exceeding 50 kW: {len(high_power_indices)} intervals")
if len(high_power_indices) > 0:
    print(f"Time range: {high_power_hours[0]:.1f} - {high_power_hours[-1]:.1f} hours")

# 6. Calculate cost
rate = 0.12  # $/kWh
total_cost = total_energy * rate

print(f"\nElectricity cost at ${rate}/kWh: ${total_cost:.2f}")

# Additional analysis
print("\nLoad profile summary:")
for hour in [0, 6, 8, 12, 19, 22]:
    idx = int(hour * 12)
    print(f"Hour {hour:2d}: {power_kw[idx]:6.1f} kW")

Power System Monitoring Analysis
Monitoring period: 24 hours
Sample interval: 5 minutes
Total samples: 288

Total energy consumed: 597.4 kWh
Average power: 24.9 kW
Peak power: 53.9 kW at hour 18.1

Periods exceeding 50 kW: 25 intervals
Time range: 18.1 - 21.0 hours

Electricity cost at $0.12/kWh: $71.69

Load profile summary:
Hour  0:   14.2 kW
Hour  6:   21.1 kW
Hour  8:   40.4 kW
Hour 12:   23.2 kW
Hour 19:   52.1 kW
Hour 22:   12.0 kW


## Summary

This lesson has covered fundamental NumPy concepts essential for power system data analysis:

**Array Creation and Manipulation**: We learned how to create arrays from measurements, perform unit conversions, and work with multi-dimensional data structures that represent time series and spatial data.

**Vectorized Operations**: NumPy's ability to operate on entire arrays eliminates loops and provides significant performance improvements. Broadcasting enables operations between arrays of different shapes.

**Statistical Analysis**: Built-in functions for mean, standard deviation, and other statistics make it easy to analyze power system measurements and identify patterns or anomalies.

**Complex Numbers**: Native support for complex arithmetic is essential for AC circuit analysis, power calculations, and impedance computations.

**Matrix Operations**: Linear algebra operations provide the foundation for solving circuit equations and analyzing network relationships.

These NumPy skills form the foundation for more advanced topics in data analysis, optimization, and numerical methods that you'll encounter in subsequent modules.

```{admonition} Next Steps
:class: tip

To reinforce these concepts:
1. Practice with real measurement data from power system sensors
2. Explore NumPy's documentation for additional functions
3. Combine NumPy with matplotlib for data visualization
4. Apply these techniques to your own power system analysis problems
```